In [ ]:
from loader import Loader
l = Loader()

l.load_case_verdicts()
df = l.case_verdict_df.copy()
# l.load_case_data()
# df = l.cases_df.copy()


In [ ]:
import pandas as pd
from constants import *

df['gerechtscode'] = pd.DataFrame(df['case_id'].str.split('-').tolist(), columns=['a', 'b', 'code', 'd', 'e'])['code']
df['instantie'] = df['gerechtscode'].map(GERECHTSCODE_INSTANTIE_MAPPING)
df['locatie'] = pd.DataFrame(df['instantie'].str.split(' ', 2).tolist(), columns=['a', 'locatie', 'rest'])['locatie']

for key, val in EXTRA_MAPPER.items():
    df.loc[df['instantie'] == key, 'locatie'] = val

# df['provincie'] = df['locatie'].map(LOCATIE_PROVINCIE_MAPPER)

In [ ]:
for i, row in df.iterrows():
    if 'peek' in row['procedure_soorten'].lower():
        print(i)

df.iloc[13515]

In [ ]:
df.procedure_soorten = df.procedure_soorten.str.replace("Peek,", "")
df.procedure_soorten = df.procedure_soorten.apply(lambda x: x[1:-1].replace('"', "").split(','))
df.rechtsgebieden = df.rechtsgebieden.apply(lambda x: x[1:-1].replace('"', "").split(','))

df = df.explode('procedure_soorten').reset_index(drop=True)
df = df.explode('rechtsgebieden').reset_index(drop=True)


In [ ]:
df['rechtsgebieden'].value_counts()
df['instantie'].value_counts()
df['procedure_soorten'].value_counts()

In [ ]:
for drug, drug_ingredients in DRUGS_AND_PRECURSORS_TABLEAU.items():
    print(drug)
    print(drug_ingredients['altnames'])
    # drug_counts = drug_ingredients['altnames'].counting-stuff
    for precursor, precursor_info in drug_ingredients['precursors'].items():
        print(precursor)
        print(precursor_info['altnames'])
        # precursor_counts = precursor_info['altnames'].counting-stuff
        for ppc, lvals in precursor_info['preprecursors'].items():
            print(f"{ppc:10}")
            print(              lvals)
    print('-----------------------')
    print('-----------------------')
    print('')

In [ ]:
from loader import Loader
l = Loader()


In [ ]:
from loader import Loader
from sqlalchemy import create_engine
l = Loader()

l.propagate_count_changes()

cols_to_drop = [col for col in l.drug_prevalence_count_tableau.columns if '_y' in col]
cols_rename = {col: col[:-2] for col in l.drug_prevalence_count_tableau.columns if '_x' in col}
l.drug_prevalence_count_tableau.drop(columns=cols_to_drop, inplace=True)
l.drug_prevalence_count_tableau.rename(columns=cols_rename, inplace=True)

df = l.drug_prevalence_count_tableau.sum(axis=0).reset_index()
df.rename(columns={'index': 'col', 0: 'sum'}, inplace=True)

cols_to_drop = df[df['sum'] == 0]['col'].tolist()
l.drug_prevalence_count_tableau.drop(columns=cols_to_drop, inplace=True)

engine = create_engine(l.connection_str)
conn = engine.connect()
l.drug_prevalence_count_tableau.to_sql('drug_prevalence_count_tableau', con=conn, schema='public', if_exists='replace', index=False, chunksize=500)
conn.close()

In [ ]:
l.propagate_count_changes()

In [ ]:
l.drug_prevalence_count_tableau.head()

In [ ]:
l.drug_prevalence_count_tableau.columns

In [ ]:
cols_to_drop = [col for col in l.drug_prevalence_count_tableau.columns if '_y' in col]
cols_rename = {col: col[:-2] for col in l.drug_prevalence_count_tableau.columns if '_x' in col}
l.drug_prevalence_count_tableau.drop(columns=cols_to_drop, inplace=True)
l.drug_prevalence_count_tableau.rename(columns=cols_rename, inplace=True)

In [ ]:
df = l.drug_prevalence_count_tableau.sum(axis=0).reset_index()
df.rename(columns={'index': 'col', 0: 'sum'}, inplace=True)

In [ ]:
cols_to_drop = df[df['sum'] == 0]['col'].tolist()
l.drug_prevalence_count_tableau.drop(columns=cols_to_drop, inplace=True)